
## Overview

This notebook will provide analyses and solutions to a set of questions related to the imported data.


In [0]:
# +++++++++++++++++++++++++++++++++++++++++++
#                 UTILS
# +++++++++++++++++++++++++++++++++++++++++++
def compare_dfs_on_one_column(df1, df2, col1_name, col2_name, derived_col_name): # Questions 1
    col_df1 = df1.select(col1_name).distinct().sort(col1_name)
    col_df2 = df2.select(col2_name).distinct().sort(col2_name).withColumnRenamed(col2_name, derived_col_name)
    comparison_df = col_df1.join(col_df2, col_df1[col1_name] == col_df2[derived_col_name], "outer")
    return(comparison_df)

def get_column_values(df, colName):
    return df.select(colName).distinct()

def find_nulls(df, colName):
    null_df = df.filter(col(colName).isNull()) 
    if null_df.count() == 0:
        print("No Nulls Found for column: " +  colName)
        return null_df
    return null_df
    


In [0]:
from pyspark.sql.window import Window	
from pyspark.sql.functions import *

# File locations and type
uni_file_path = "/FileStore/tables/University_Data.csv"
admissions_file_path = "/FileStore/tables/College_Admissions.csv"
college_data_file_path = "/FileStore/tables/College_Data.csv"
global_ranking_file_path = "/FileStore/tables/Universities_Global_Ranking.csv"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","
# ========================================================================================================================================================
#                                                                     IMPORT DATA
# ========================================================================================================================================================
# The applied options are for CSV files. For other file types, these will be ignored.
university_df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(uni_file_path)

college_admissions_df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(admissions_file_path)

college_data_df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(college_data_file_path)

universities_global_df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(global_ranking_file_path)


# ==================================================================================================================================================================
#                                                                        ANALYSIS
# ==================================================================================================================================================================
# ...............................................................................................................................
#                    PROBLEM 1) show university trend by global ranking and number of applicants vs admissions 
# ...............................................................................................................................
# - (College_Admissions csv + Global Ranking csv)
def compare_dfs_on_one_column(df1, df2, col1_name, col2_name): # Questions 1
    col_df1 = df1.select(col1_name).distinct().sort(col1_name)
    col_df2 = df2.select(col2_name).distinct().sort(col2_name)
    comparison_df = col_df1.join(col_df2, col_df1[col1_name] == col_df2[col2_name], "outer")
    return(comparison_df)

# +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
#  STEP 1: Remove irrelevant data/ Select Only Relevant Data
# +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
# * * It's good practice to define your schema before importing your data, but decided against that since I wasn't comfortable with the data. * *
world_rankings = universities_global_df.select(['world_rank', 'institution',"country", "national_rank", "year"]).distinct()
college_admissions_applicants = college_admissions_df.select(["Name", "Applicants total","Admissions total", "Enrolled total", "State abbreviation"])

# ++++++++++++++++++++++++++++++++++++++++++++++++
#  STEP 2: Check For & Remove Duplicates
# ++++++++++++++++++++++++++++++++++++++++++++++++
world_rankings_dupe_check = world_rankings.select(['world_rank', 'institution',"country", "national_rank", "year"]).distinct()
college_admissions_applicants_dupe_check = college_admissions_applicants.select(["Name", "Applicants total","Admissions total", "Enrolled total", "State abbreviation"]).distinct()
print("Num dupes in world_rankings_df: " , world_rankings.count() - world_rankings_dupe_check.count())
print("Num dupes in world_rankings_df: " , college_admissions_applicants.count() - college_admissions_applicants_dupe_check.count())


# +++++++++++++++++++++++++++++++++++++++
#  STEP 3: Clean Data/Check for nulls
# +++++++++++++++++++++++++++++++++++++++
#   * * fillna() could be used to replace null values with a value I deem approp. Also 'DROPMALFORMED' arg in df import * *
        ### Check for/Drop Nulls
print("---   CHECK FOR NULLS   ---")
print("University Global Rankings")
find_nulls(universities_global_df,"institution")
find_nulls(universities_global_df,"world_rank")
find_nulls(universities_global_df,"national_rank")
find_nulls(universities_global_df,"year")

print("University Data")
university_df = university_df.na.drop(subset=["world_rank"])
university_df = university_df.na.drop(subset=["school"])
university_df = university_df.na.drop(subset=["country"])
university_df = university_df.na.drop(subset=["year"])
find_nulls(university_df,"world_rank")
find_nulls(university_df,"school")
find_nulls(university_df,"country")
find_nulls(university_df,"year")

print("College Admissions")
college_admissions_df = college_admissions_df.na.drop(subset=["Applicants total"])
college_admissions_df = college_admissions_df.na.drop(subset=["Admissions total"])
college_admissions_df = college_admissions_df.na.drop(subset=["Enrolled total"])
find_nulls(college_admissions_df,"Name")
find_nulls(college_admissions_df,"Applicants total")
find_nulls(college_admissions_df,"Admissions total")
find_nulls(college_admissions_df,"Enrolled total")
find_nulls(college_admissions_df,"State abbreviation")

print("College Data")
college_admissions_applicants = college_admissions_applicants.na.drop(subset=["Applicants total"])
college_admissions_applicants = college_admissions_applicants.na.drop(subset=["Admissions total"])
college_admissions_applicants = college_admissions_applicants.na.drop(subset=["Enrolled total"])
find_nulls(college_admissions_applicants,"Name")
find_nulls(college_admissions_applicants,"Applicants total")
find_nulls(college_admissions_applicants,"Admissions total")
find_nulls(college_admissions_applicants,"Enrolled total")
find_nulls(college_admissions_applicants,"State abbreviation")

        ### Data Scrub Re-visit - "living section" for errors I have found while working
#### Removing quotes from column values in relevant dataframes
world_rankings = world_rankings.withColumn("institution", regexp_replace("institution", "\"", ""))
        
        # +++++++++++++++++++++++++++++++++++++++
        #  STEP 3a: Check data purview
        # +++++++++++++++++++++++++++++++++++++++

                # display(university_df.select("country").distinct())
                # display(world_rankings.select("country").distinct())
                # display(college_admissions_applicants.select("country").distinct())



# +++++++++++++++++++++++++++++++++++++++
#  STEP 4: Check For Missing Data
# +++++++++++++++++++++++++++++++++++++++
# Combine all files to see if it fills in the missing colleges between college_admissions and world_rankings and brings us to 1000
# # Combining University df fills in some missing data. Brings shared colleges from 120 -> 258  
print("")
print("-----------------------------------------------------")
print("---   CHECK FOR MISSING DATA WHEN COMBINING DFS   ---")
print("---   Here, we're combining all of the dataframes.  ---")
print("world_rankings df college count BEFORE MERGE: ",  world_rankings.select("institution").count())
print("college_admissions_applicants df college count BEFORE MERGE: " , college_admissions_applicants.select("Name").count())

university_rankings = university_df.select(["world_rank", "school","country", "year"]).withColumnRenamed("school", "institution").distinct()
world_rankings_combined = world_rankings.select(['world_rank', 'institution',"country", "year"]).union(university_rankings)
world_rankings_combined = world_rankings_combined.select(['world_rank', 'institution',"country", "year"]).distinct()
world_rankings_combined = world_rankings_combined.withColumn("world_rank", world_rankings_combined.world_rank.cast("integer"))
world_rankings_combined = world_rankings_combined.filter(world_rankings_combined.world_rank.isNotNull())

college_admissions_applicants2 = college_data_df.select(["_c0", "Apps","Accept", "Enroll"]).withColumnRenamed("_c0", "Name").distinct()
combined_college_admissions_applicants = college_admissions_applicants.select(["Name", "Applicants total","Admissions total", "Enrolled total"]).union(college_admissions_applicants2)
combined_college_admissions_applicants = combined_college_admissions_applicants.select(["Name", "Applicants total","Admissions total", "Enrolled total"]).distinct()

print("world_rankings df college count AFTER MERGE: ",  world_rankings_combined.select(["institution"]).count())
print("college_admissions_applicants df college count AFTER MERGE: " , combined_college_admissions_applicants.select("Name").count())
missing_colleges_check = world_rankings_combined.select("institution").distinct().join(combined_college_admissions_applicants.select("Name").distinct(),world_rankings_combined.institution == combined_college_admissions_applicants.Name, "inner")

        ### Validate New Merged Dataframe:
print("---   CHECK FOR DISCREPANCIES IN COMBNINED DATAFRAMES   ---")

data_discrepancy_check = (world_rankings_combined.select("*").join(world_rankings_combined.groupBy("institution").count(), "institution")).sort("institution", "year")
print("Original Global Rank DF Distinct Colleges ('Universities_Global_Ranking.csv') | Merged DF Distinct Colleges ('Universities_Global_Ranking.csv' + 'Universities_Data.csv')")
display(data_discrepancy_check)
print("snapshot of the discrepancy in the data")
University_Data_Example_Sample = university_df.select(["world_rank", "school","country", "year"]).where(col("school") == "Arizona State University").sort("year")
parquet_file_import_test_1 = spark.read.parquet("dbfs:/University_Data_Example_Sample.parquet")
display(parquet_file_import_test_1)
University_Global_Ranking_Example_Sample = world_rankings.select(["world_rank", "institution","country", "year"]).where(col("institution") == "Arizona State University").sort("year")
parquet_file_import_test_2 = spark.read.parquet("dbfs:/University_Global_Ranking_Example_Sample.parquet")
display(parquet_file_import_test_2)

# +++++++++++++++++++++++++++++++++++++++
#  STEP 5: Craft Solution
# +++++++++++++++++++++++++++++++++++++++

# (A) Generalize on subsets of 10. Avg Top 10, Top 20, etc
# colleges_averaged_rank_preprocess = world_rankings.select("institution", "world_rank", "national_rank").join(world_rankings.groupBy("institution").avg("world_rank", "national_rank"), "institution")

# (B) Get a general distinct ranking for each American College - Generate final score by averaging the score for every year -> Export as parquet
        # (TEST SAMPLE) 
        # aggregate_sample = world_rankings.select("institution", "world_rank", "national_rank", "year").where(col("institution").isin(["Duke University", "Harvard University", "Georgia Institute of Technology"])).sort("institution")
        # num_colleges_reports = world_rankings.select("institution", "world_rank", "national_rank", "year").join(world_rankings.groupBy("institution").count(), "institution")
        # colleges_not_recorded_all_4_years_DUPLICATES = num_colleges_reports.where(col("count") < 4).sort("institution", ascending=True)
        # colleges_not_recorded_all_4_samyears_NO_DUPES = colleges_not_recorded_all_4_years_DUPLICATES.select("institution", "count").distinct().sort("count", ascending=False)
        # colleges_not_recorded_all_4_years_DUPLICATES.write.parquet("colleges_not_recorded_all_4_years_DUPLICATES.parquet")
        # colleges_not_recorded_all_4_years_NO_DUPES.write.parquet("colleges_not_recorded_all_4_years_NO_DUPES.parquet")

# (B1) CALCULATING STANDARD DEVIATION IN RANKS OVER THE YEARS THEY'RE DOCUMENTED
print("-----------------------------------------------------")
print("CALCULATING STANDARD DEVIATION IN RANKS OVER THE YEARS THEY'RE DOCUMENTED")
stddevs_for_world_ranks_by_uni = world_rankings.select("institution", "world_rank", "national_rank").join(world_rankings.groupBy("institution").agg(stddev("world_rank")), "institution")
# display(stddevs_for_world_ranks_by_uni.select("institution").count())
stddevs_for_both_ranks_by_uni = stddevs_for_world_ranks_by_uni.join(world_rankings.groupBy("institution").agg(stddev("national_rank")), "institution")
# display(stddevs_for_both_ranks_by_uni.select("institution").count())
stddevs_for_both_ranks_plus_count = stddevs_for_both_ranks_by_uni.join(world_rankings.groupBy("institution").count(), "institution")
stddevs_for_both_ranks_by_uni = stddevs_for_both_ranks_by_uni.select("institution", "stddev_samp(world_rank)", "stddev_samp(national_rank)").distinct().sort("stddev_samp(world_rank)", ascending=False)
display(stddevs_for_both_ranks_by_uni)
# stddevs_for_both_ranks_by_uni.write.parquet("stddevs_for_both_ranks_by_uni.parquet")
# display(stddevs_for_both_ranks_plus_count.select("institution", "stddev_samp(world_rank)", "stddev_samp(national_rank)", "count").distinct().sort("stddev_samp(world_rank)", ascending=True).where(col("count") > 1)) # Nulls are only for records with one ranking.

# (B2) Calculate final score by summing all available ranks for every college. -> (ALT) Sum only the world ranks as requested *facepalm* -> (ALT ALT) -> Averages Girl. Geez.
colleges_averaged_rank_preprocess = world_rankings.select("institution", "world_rank", "national_rank").join(world_rankings.groupBy("institution").avg("world_rank", "national_rank"), "institution")
combined_colleges_averaged_rank_preprocess = world_rankings_combined.select("institution", "world_rank").join(world_rankings_combined.groupBy("institution").avg("world_rank"), "institution")
# display(colleges_averaged_rank_preprocess)

        ### Dupe Check
averaged_college_rankings_almost= colleges_averaged_rank_preprocess.select("institution","avg(world_rank)", "avg(national_rank)").distinct() 
combined_averaged_college_rankings_almost= combined_colleges_averaged_rank_preprocess.select("institution","avg(world_rank)").distinct() 
# display(averaged_college_rankings_almost)
        ### Sort
averaged_global_college_rankings_final = averaged_college_rankings_almost.select("institution", "avg(world_rank)").sort("avg(world_rank)", ascending=True) 
combined_averaged_global_college_rankings_final = combined_averaged_college_rankings_almost.select("institution", "avg(world_rank)").sort("avg(world_rank)", ascending=True) 
# display(averaged_global_college_rankings_final)

# +++++++++++++++++++++++++++++++++++++++
#  STEP 6: Validate Data
# +++++++++++++++++++++++++++++++++++++++
# Check final score values by checking sample against hand-calculated expected values 
        # test_aggregate_sample = averaged_global_college_rankings_final.where(col("institution").isin(["Duke University", "Harvard University", "Georgia Institute of Technology"]))
applicants_BY_world_rank_df = college_admissions_applicants.join(averaged_global_college_rankings_final, college_admissions_applicants.Name == averaged_global_college_rankings_final.institution).sort("avg(world_rank)", ascending=True)
applicants_BY_world_rank_df_ALL_FILES = combined_college_admissions_applicants.join(combined_averaged_global_college_rankings_final, combined_college_admissions_applicants.Name == combined_averaged_global_college_rankings_final.institution).sort("avg(world_rank)", ascending=True)
# applicants_BY_world_rank_df.write.parquet("applicants_admissions_BY_world_rank_df_all_files_combined.parquet")
# applicants_BY_world_rank_df.write.format("parquet")\
#         .mode("overwrite")\
#         .save("applicants_admissions_BY_world_rank_df_all_files_combined.parquet")
# Check to see if any colleges are missing
print("Combine College_Admissions.csv df with the average score for each valid college in University_Global_ranking.csv")
print("Final df college count: ",  applicants_BY_world_rank_df.select("institution").count())
print("OG df college count: " , world_rankings.select("institution").count())


display(applicants_BY_world_rank_df)
display(applicants_BY_world_rank_df_ALL_FILES)


# # # (B2) Generate score using Bayesian method -> No NEED. Would need to reframe problem as probability. Interesting but Unecessary. 
# ...............................................................................................................................
#                    PROBLEM 2) Provide the trends by state with respect to the number of applicants vs admissions per year  
# ...............................................................................................................................
# - (ALL csv files)
# -> The point I realized that my dataset is concerningly smaller than expected. :( --> Revisit Step 4, uncomment, and rerun.
        # print("world_rankings df college count: ",  world_rankings.select("institution").count())
        # print("college_admissions_applicants df college count: " , college_admissions_applicants.select("Name").count())
        #         # # display(compare_dfs_on_one_column(world_rankings,college_admissions_applicants, "institution", "Name"))
        # missing_colleges_check = world_rankings.select("institution").join(college_admissions_applicants.select("Name"),world_rankings.institution == college_admissions_applicants.Name, "left")
        # display("missing_colleges_check", missing_colleges_check.count())
        # display(missing_colleges_check)             
        
# ...............................................................................................................................
#                    PROBLEM 3) Provide the trends by state with respect to the number of applicants vs admissions per year
# ...............................................................................................................................
# - (College_Admissions csv + Global Ranking csv)
applicants_BY_world_rank_state_analysis_df = (applicants_BY_world_rank_df.groupBy("State abbreviation").avg("avg(world_rank)", "Applicants total", "Admissions total")).sort("avg(avg(world_rank))")
# applicants_BY_world_rank_state_analysis_df.write.parquet("applicants_BY_world_rank_BY_state_analysis_.parquet")

display(applicants_BY_world_rank_state_analysis_df)


# - (College_Admissions csv + Global Ranking csv) SAT scores/ Student Aid/Associates degrees per global ranking

# - (College_Admissions csv + College Data csv) How do numApplicants/numAdmissions vary by state?How is price related to this? How is financial aid influencing this?



Num dupes in world_rankings_df:  0
Num dupes in world_rankings_df:  0
---   CHECK FOR NULLS   ---
University Global Rankings
No Nulls Found for column: institution


world_rank,institution,country,national_rank,quality_of_education,alumni_employment,quality_of_faculty,publications,influence,citations,broad_impact,patents,score,year


No Nulls Found for column: world_rank


world_rank,institution,country,national_rank,quality_of_education,alumni_employment,quality_of_faculty,publications,influence,citations,broad_impact,patents,score,year


No Nulls Found for column: national_rank


...

Name,Applicants total,Admissions total,Enrolled total,State abbreviation,institution,avg(world_rank)
Harvard University,35023,2047,1659,Massachusetts,Harvard University,1.0
Stanford University,38828,2208,1677,California,Stanford University,2.25
Massachusetts Institute of Technology,18989,1548,1115,Massachusetts,Massachusetts Institute of Technology,3.0
Princeton University,26499,1963,1285,New Jersey,Princeton University,8.0
University of Chicago,30304,2670,1426,Illinois,University of Chicago,9.0
Yale University,28977,2043,1356,Connecticut,Yale University,9.75
California Institute of Technology,5535,584,249,California,California Institute of Technology,10.0
Cornell University,39999,6222,3223,New York,Cornell University,11.5
University of Pennsylvania,31282,3830,2421,Pennsylvania,University of Pennsylvania,13.25
Johns Hopkins University,21336,3869,1390,Maryland,Johns Hopkins University,16.75


Databricks visualization. Run in Databricks to view.

Name,Applicants total,Admissions total,Enrolled total,institution,avg(world_rank)
Harvard University,13865,2165,1606,Harvard University,2.1
Harvard University,35023,2047,1659,Harvard University,2.1
Stanford University,38828,2208,1677,Stanford University,2.8
Massachusetts Institute of Technology,6411,2140,1078,Massachusetts Institute of Technology,4.3
Massachusetts Institute of Technology,18989,1548,1115,Massachusetts Institute of Technology,4.3
Columbia University,6756,1930,871,Columbia University,6.75
Princeton University,13218,2042,1153,Princeton University,8.0
Princeton University,26499,1963,1285,Princeton University,8.0
University of Chicago,6348,2999,922,University of Chicago,9.7
University of Chicago,30304,2670,1426,University of Chicago,9.7


State abbreviation,avg(avg(world_rank)),avg(Applicants total),avg(Admissions total)
Maryland,16.75,21336.0,3869.0
New Hampshire,57.0,23110.0,2260.0
Indiana,70.5,17647.0,3936.0
Massachusetts,141.26041666666666,25613.0,6882.875
Iowa,158.75,19090.5,15505.5
Connecticut,188.08333333333334,22167.333333333332,6296.666666666667
Pennsylvania,194.3,25102.6,12071.2
California,225.5,34869.2,9645.4
Texas,236.0625,18537.75,8920.5
Colorado,250.875,12741.25,9238.75


Databricks visualization. Run in Databricks to view.